In [2]:
import pandas as pd
import geopandas as gpd
import os
import numpy as np
import rasterio
from rasterio.features import rasterize
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

In [3]:
pd.set_option('display.max_rows', 10)

***Estimate the amount of kgCO2-eq emitted from the production of cement used in construction of all residential buildings in Norway untill 2025.*** For simplicity, I am multiplying the amount of cement used in each municipality, per building archetype, and per year, by the CO₂ emission factor (which is 0.7-0.9 kg CO₂-eq per kg of cement). This will give me the total emissions from cement production in residential buildings across Norway until 2025 base on a certain assumption of proximation of emission factor.

In [4]:
data = pd.read_csv('/Users/dolgayamaria/Thesis/Practical Part/Scripts/Output/final_merge_for_LCA.csv') 

In [5]:
cement_data = data[data["Material type"] == "cement"]
cement_data

,kommunenum,kommunenav,total_heated_area,Archetype,DB_Area,DB_Basement,DB_total,Material type,Value,Material_intensity,total_material_amounts
0,4216,Birkenes,224.0,AB_1955,728.7,155.4,884.1,cement,76222.192629,104.600237,23430.453066
10,4216,Birkenes,0.0,AB_1956_1970,1326.3,277.9,1604.2,cement,119647.450292,90.211453,0.000000
20,4216,Birkenes,0.0,AB_1971_1980,1763.6,464.0,2227.6,cement,60638.402586,34.383308,0.000000
30,4216,Birkenes,1408.0,AB_1981_1990,1853.6,481.5,2335.1,cement,69830.821810,37.673080,53043.697189
40,4216,Birkenes,2610.0,AB_1991_2000,1673.2,438.8,2112.0,cement,57112.478979,34.133683,89088.913540
...,...,...,...,...,...,...,...,...,...,...,...
82774,1871,Andøy,1440.0,MFH_1981_1990,338.2,122.8,461.0,cement,7069.510689,20.903343,30100.814289
82784,1871,Andøy,1615.0,MFH_1991_2000,293.8,107.4,401.2,cement,10444.497800,35.549686,57412.743185
82794,1871,Andøy,2464.0,MFH_2001_2010,301.4,111.1,412.5,cement,6454.943430,21.416534,52770.340446
82804,1871,Andøy,0.0,MFH_2011_2020,386.3,141.7,528.0,cement,6311.533101,16.338424,0.000000


In [6]:
cement_data["CO2_emissions_07"] = cement_data["total_material_amounts"]*0.7
cement_data["CO2_emissions_09"] = cement_data["total_material_amounts"]*0.9
cement_data

/var/folders/kn/87q7q5h140ldb97ht3jdcpr00000gn/T/ipykernel_81491/2635423361.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cement_data["CO2_emissions_07"] = cement_data["total_material_amounts"]*0.7
/var/folders/kn/87q7q5h140ldb97ht3jdcpr00000gn/T/ipykernel_81491/2635423361.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cement_data["CO2_emissions_09"] = cement_data["total_material_amounts"]*0.9


,kommunenum,kommunenav,total_heated_area,Archetype,DB_Area,DB_Basement,DB_total,Material type,Value,Material_intensity,total_material_amounts,CO2_emissions_07,CO2_emissions_09
0,4216,Birkenes,224.0,AB_1955,728.7,155.4,884.1,cement,76222.192629,104.600237,23430.453066,16401.317146,21087.407759
10,4216,Birkenes,0.0,AB_1956_1970,1326.3,277.9,1604.2,cement,119647.450292,90.211453,0.000000,0.000000,0.000000
20,4216,Birkenes,0.0,AB_1971_1980,1763.6,464.0,2227.6,cement,60638.402586,34.383308,0.000000,0.000000,0.000000
30,4216,Birkenes,1408.0,AB_1981_1990,1853.6,481.5,2335.1,cement,69830.821810,37.673080,53043.697189,37130.588032,47739.327470
40,4216,Birkenes,2610.0,AB_1991_2000,1673.2,438.8,2112.0,cement,57112.478979,34.133683,89088.913540,62362.239478,80180.022186
...,...,...,...,...,...,...,...,...,...,...,...,...,...
82774,1871,Andøy,1440.0,MFH_1981_1990,338.2,122.8,461.0,cement,7069.510689,20.903343,30100.814289,21070.570002,27090.732860
82784,1871,Andøy,1615.0,MFH_1991_2000,293.8,107.4,401.2,cement,10444.497800,35.549686,57412.743185,40188.920229,51671.468866
82794,1871,Andøy,2464.0,MFH_2001_2010,301.4,111.1,412.5,cement,6454.943430,21.416534,52770.340446,36939.238312,47493.306402
82804,1871,Andøy,0.0,MFH_2011_2020,386.3,141.7,528.0,cement,6311.533101,16.338424,0.000000,0.000000,0.000000


In [7]:
total_CO2_07 = cement_data["CO2_emissions_07"].sum()
total_CO2_09 = cement_data["CO2_emissions_09"].sum()
print(f"Total CO2 emissions from cement productions across Norway are {total_CO2_07}-{total_CO2_09} kg, or {(total_CO2_07/1000000000).round(3)}-{(total_CO2_09/1000000000).round(3)} Mt")



Total CO2 emissions from cement productions across Norway are 2807866871.049714-3610114548.49249 kg, or 2.808-3.61 Mt


## Now, let's use the data from ODYMRECC (4_EI_ProcessEnergyIntensity_V3.2).

**CEMENT**

production of cement - electricity - 2025 - 0.3276 MJ/kg

production of cement - natural gas - 2025 - 2.94 MJ/kg

In [8]:
cement_data["E_intensity_electr"] = cement_data["total_material_amounts"]*0.3276
cement_data["E_intensity_NatGas"] = cement_data["total_material_amounts"]*2.94

/var/folders/kn/87q7q5h140ldb97ht3jdcpr00000gn/T/ipykernel_81491/2415305971.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cement_data["E_intensity_electr"] = cement_data["total_material_amounts"]*0.3276
/var/folders/kn/87q7q5h140ldb97ht3jdcpr00000gn/T/ipykernel_81491/2415305971.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cement_data["E_intensity_NatGas"] = cement_data["total_material_amounts"]*2.94


carbon intensity of electricity in Norway - 28 gCO2-eq/kWh (In Scarlat et al. (2022)) ≈0.00778kgCO₂-eq/MJ

carbon intensity of natural gas - 0.07 kgCO2-eq/MJ (ODYMRECC) 

carbon intensity of diesel - 0.07 kgCO2-eq/MJ (ODYMRECC) 

In [9]:
cement_data["Emissions_electr"] = cement_data["E_intensity_electr"]*0.00778
cement_data["Emissions_NatGas"] = cement_data["E_intensity_NatGas"]*0.07


/var/folders/kn/87q7q5h140ldb97ht3jdcpr00000gn/T/ipykernel_81491/2377604127.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cement_data["Emissions_electr"] = cement_data["E_intensity_electr"]*0.00778
/var/folders/kn/87q7q5h140ldb97ht3jdcpr00000gn/T/ipykernel_81491/2377604127.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cement_data["Emissions_NatGas"] = cement_data["E_intensity_NatGas"]*0.07


In [10]:
emissions_electr = cement_data["Emissions_electr"].sum()
emissions_NatGas = cement_data["Emissions_NatGas"].sum()

print(f"Total CO2 emissions from cement productions using electricity across Norway are {emissions_electr} kg, or {(emissions_electr/1000000000).round(3)} Mt")
print(f"Total CO2 emissions from cement productions using natural gas across Norway are {emissions_NatGas} kg, or {(emissions_NatGas/1000000000).round(3)} Mt")


Total CO2 emissions from cement productions using electricity across Norway are 10223555.592166848 kg, or 0.01 Mt
Total CO2 emissions from cement productions using natural gas across Norway are 825512860.0886161 kg, or 0.826 Mt


**CONCRETE AND AGGREGATES**

production of concrete - electricity - 2025 - 0,00828 MJ/kg

production of concrete - diesel - 2025 - 0,022971 MJ/kg



In [11]:
concrete_data = data[data["Material type"] == "concrete"]
concrete_data["E_intensity_electr"] = concrete_data["total_material_amounts"]*0.00828
concrete_data["E_intensity_diesel"] = concrete_data["total_material_amounts"]*0.022971

concrete_data["Emissions_electr"] = concrete_data["E_intensity_electr"]*0.00778
concrete_data["Emissions_diesel"] = concrete_data["E_intensity_diesel"]*0.07

emissions_electr = concrete_data["Emissions_electr"].sum()
emissions_diesel = concrete_data["Emissions_diesel"].sum()

print(f"Total CO2 emissions from concrete productions using electricity across Norway are {emissions_electr} kg, or {(emissions_electr/1000000000).round(3)} Mt")
print(f"Total CO2 emissions from concrete productions using diesel across Norway are {emissions_diesel} kg, or {(emissions_diesel/1000000000).round(3)} Mt")


Total CO2 emissions from concrete productions using electricity across Norway are 2994900.1609323677 kg, or 0.003 Mt
Total CO2 emissions from concrete productions using diesel across Norway are 74756740.4930023 kg, or 0.075 Mt


/var/folders/kn/87q7q5h140ldb97ht3jdcpr00000gn/T/ipykernel_81491/3409822191.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  concrete_data["E_intensity_electr"] = concrete_data["total_material_amounts"]*0.00828
/var/folders/kn/87q7q5h140ldb97ht3jdcpr00000gn/T/ipykernel_81491/3409822191.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  concrete_data["E_intensity_diesel"] = concrete_data["total_material_amounts"]*0.022971
/var/folders/kn/87q7q5h140ldb97ht3jdcpr00000gn/T/ipykernel_81491/3409822191.py:5: S

**WOOD AND WOOD PRODUCTS**

production of wood - electricity - 2025 - 0,520615384615385 MJ/kg

production of wood - diesel - 2025 - 0,109692307692308 MJ/kg



In [12]:
wood_data = data[data["Material type"] == "wood and wood products"]
wood_data["E_intensity_electr"] = wood_data["total_material_amounts"]*0.520615384615385 
wood_data["E_intensity_diesel"] = wood_data["total_material_amounts"]*0.109692307692308 

wood_data["Emissions_electr"] = wood_data["E_intensity_electr"]*0.00778
wood_data["Emissions_diesel"] = wood_data["E_intensity_diesel"]*0.07

emissions_electr = wood_data["Emissions_electr"].sum()
emissions_diesel = wood_data["Emissions_diesel"].sum()

print(f"Total CO2 emissions from wood and wood products productions using electricity across Norway are {emissions_electr} kg, or {(emissions_electr/1000000000).round(3)} Mt")
print(f"Total CO2 emissions from wood and wood products productions using diesel across Norway are {emissions_diesel} kg, or {(emissions_diesel/1000000000).round(3)} Mt")


Total CO2 emissions from wood and wood products productions using electricity across Norway are 173839585.96378115 kg, or 0.174 Mt
Total CO2 emissions from wood and wood products productions using diesel across Norway are 329553780.0541918 kg, or 0.33 Mt


/var/folders/kn/87q7q5h140ldb97ht3jdcpr00000gn/T/ipykernel_81491/1793900708.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wood_data["E_intensity_electr"] = wood_data["total_material_amounts"]*0.520615384615385
/var/folders/kn/87q7q5h140ldb97ht3jdcpr00000gn/T/ipykernel_81491/1793900708.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wood_data["E_intensity_diesel"] = wood_data["total_material_amounts"]*0.109692307692308
/var/folders/kn/87q7q5h140ldb97ht3jdcpr00000gn/T/ipykernel_81491/1793900708.py:5

**PLASTICS**

production of plastics - electricity - 2025 - 3,94 MJ/kg

production of plastics - natural gas - 2025 - 5,67 MJ/kg

production of plastics - diesel - 2025 - 7,03 MJ/kg



**CONSTRUCTION GRADE STEEL**

production of construction grade steel - electricity - 2025 - 3,8268 MJ/kg

production of construction grade steel - hydrogen - 2025 - 6,848856 MJ/kg

In [13]:
steel_data = data[data["Material type"] == "construction grade steel"]
steel_data["E_intensity_electr"] = steel_data["total_material_amounts"]*3.8268 
steel_data["E_intensity_hydrogen"] = steel_data["total_material_amounts"]*0.109692307692308 

steel_data["Emissions_electr"] = steel_data["E_intensity_electr"]*0.00778
# steel_data["Emissions_hydrogen"] = steel_data["E_intensity_hydrogen"]*0.07

emissions_electr = steel_data["Emissions_electr"].sum()
# emissions_hydrogen = steel_data["Emissions_hydrogen"].sum()

print(f"Total CO2 emissions from wood and wood products productions using electricity across Norway are {emissions_electr} kg, or {(emissions_electr/1000000000).round(3)} Mt")
# print(f"Total CO2 emissions from wood and wood products productions using diesel across Norway are {emissions_hydrogen} kg, or {(emissions_hydrogen/1000000000).round(3)} Mt")


Total CO2 emissions from wood and wood products productions using electricity across Norway are 327876666.5382905 kg, or 0.328 Mt


/var/folders/kn/87q7q5h140ldb97ht3jdcpr00000gn/T/ipykernel_81491/2870703264.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  steel_data["E_intensity_electr"] = steel_data["total_material_amounts"]*3.8268
/var/folders/kn/87q7q5h140ldb97ht3jdcpr00000gn/T/ipykernel_81491/2870703264.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  steel_data["E_intensity_hydrogen"] = steel_data["total_material_amounts"]*0.109692307692308
/var/folders/kn/87q7q5h140ldb97ht3jdcpr00000gn/T/ipykernel_81491/2870703264.py:5: Set

**GLASS**

production of glass - electricity - 2025 - 0,4009228 MJ/kg

production of glass - natural gas - 2025 - 8,1661644 MJ/kg




In [14]:
glass_data = data[data["Material type"] == "glass"]
glass_data["E_intensity_electr"] = glass_data["total_material_amounts"]*0.4009228 
glass_data["E_intensity_NatGas"] = glass_data["total_material_amounts"]*8.1661644

glass_data["Emissions_electr"] = glass_data["E_intensity_electr"]*0.00778
glass_data["Emissions_NatGas"] = glass_data["E_intensity_NatGas"]*0.07

emissions_electr = glass_data["Emissions_electr"].sum()
emissions_NatGas = glass_data["Emissions_NatGas"].sum()

print(f"Total CO2 emissions from wood and wood products productions using electricity across Norway are {emissions_electr} kg, or {(emissions_electr/1000000000).round(3)} Mt")
print(f"Total CO2 emissions from wood and wood products productions using diesel across Norway are {emissions_diesel} kg, or {(emissions_diesel/1000000000).round(3)} Mt")


Total CO2 emissions from wood and wood products productions using electricity across Norway are 2848867.905567741 kg, or 0.003 Mt
Total CO2 emissions from wood and wood products productions using diesel across Norway are 329553780.0541918 kg, or 0.33 Mt


/var/folders/kn/87q7q5h140ldb97ht3jdcpr00000gn/T/ipykernel_81491/880483033.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  glass_data["E_intensity_electr"] = glass_data["total_material_amounts"]*0.4009228
/var/folders/kn/87q7q5h140ldb97ht3jdcpr00000gn/T/ipykernel_81491/880483033.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  glass_data["E_intensity_NatGas"] = glass_data["total_material_amounts"]*8.1661644
/var/folders/kn/87q7q5h140ldb97ht3jdcpr00000gn/T/ipykernel_81491/880483033.py:5: SettingWithCo

**INSULATION**

production of insulation material - electricity - 2025 - 2,03 MJ/kg

production of insulation material - natural gas - 2025 - 18,8 MJ/kg

production of insulation material - diesel - 2025 - 15,71 MJ/kg

In [15]:
insulation_data = data[data["Material type"] == "insulation"]
insulation_data["E_intensity_electr"] = insulation_data["total_material_amounts"]*2.03  
insulation_data["E_intensity_NatGas"] = insulation_data["total_material_amounts"]*18.8 
insulation_data["E_intensity_diesel"] = insulation_data["total_material_amounts"]*15.71

insulation_data["Emissions_electr"] = insulation_data["E_intensity_electr"]*0.00778
insulation_data["Emissions_NatGas"] = insulation_data["E_intensity_NatGas"]*0.07
insulation_data["Emissions_diesel"] = insulation_data["E_intensity_diesel"]*0.07


emissions_electr = insulation_data["Emissions_electr"].sum()
emissions_NatGas = insulation_data["Emissions_NatGas"].sum()
emissions_diesel = insulation_data["Emissions_diesel"].sum()

print(f"Total CO2 emissions from insulation productions using electricity across Norway are {emissions_electr} kg, or {(emissions_electr/1000000000).round(3)} Mt")
print(f"Total CO2 emissions from insulation productions using natural gas across Norway are {emissions_NatGas} kg, or {(emissions_NatGas/1000000000).round(3)} Mt")
print(f"Total CO2 emissions from insulation productions using diesel across Norway are {emissions_diesel} kg, or {(emissions_diesel/1000000000).round(3)} Mt")


Total CO2 emissions from insulation productions using electricity across Norway are 84979378.69503933 kg, or 0.085 Mt
Total CO2 emissions from insulation productions using natural gas across Norway are 7080987144.166031 kg, or 7.081 Mt
Total CO2 emissions from insulation productions using diesel across Norway are 5917144044.406827 kg, or 5.917 Mt


/var/folders/kn/87q7q5h140ldb97ht3jdcpr00000gn/T/ipykernel_81491/3695275363.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  insulation_data["E_intensity_electr"] = insulation_data["total_material_amounts"]*2.03
/var/folders/kn/87q7q5h140ldb97ht3jdcpr00000gn/T/ipykernel_81491/3695275363.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  insulation_data["E_intensity_NatGas"] = insulation_data["total_material_amounts"]*18.8
/var/folders/kn/87q7q5h140ldb97ht3jdcpr00000gn/T/ipykernel_81491/3695275363.py:4: 

Let try making the process more accurate and import all the tables 

In [16]:
manufacturing_energy_int = pd.read_excel("/Users/dolgayamaria/Thesis/Practical Part/Data/LCA/4_EI_ManufacturingEnergyIntensity_V2.2.xlsx", sheet_name = "Values_Master")
process_energy_int = pd.read_excel("/Users/dolgayamaria/Thesis/Practical Part/Data/LCA/4_EI_ProcessEnergyIntensity_V3.2.xlsx", sheet_name = "Values_Master")
direct_emissions = pd.read_excel("/Users/dolgayamaria/Thesis/Practical Part/Data/LCA/6_PR_DirectEmissions_V1.2.xlsx", sheet_name = "Values_Master")

In [17]:
total_heated_area = pd.read_csv("/Users/dolgayamaria/Thesis/Practical Part/Scripts/Output/total_heated_area.csv")
total_heated_area

,Unnamed: 0,kommunenum,archetype,total_heated_area
0,0,3101,AB_1955,44912.0
1,1,3103,AB_1955,36456.0
2,2,3105,AB_1955,46032.0
3,3,3107,AB_1955,46760.0
4,4,3110,AB_1955,0.0
...,...,...,...,...
8563,8563,5628,MFH_2021_,960.0
8564,8564,5630,MFH_2021_,0.0
8565,8565,5632,MFH_2021_,0.0
8566,8566,5634,MFH_2021_,0.0


In [18]:
manufacturing_energy_int = manufacturing_energy_int[
    (manufacturing_energy_int["Manufacturing_i3"] == "manufacturing of SFH_standard") | 
    (manufacturing_energy_int["Manufacturing_i3"] == "manufacturing of MFH_standard") |
    (manufacturing_energy_int["Manufacturing_i3"] == "manufacturing of RT_standard")
]

In [19]:
manufacturing_energy_int

,Manufacturing_i3,Energy_Carriers_m6,Time,SSP_Regions_32,value,unit,stats_array_string,comment
20,manufacturing of SFH_standard,diesel,2010,World,6.20,MJ/m2,none,ref. [2]: value compiled from ecoinvent v3.4 b...
21,manufacturing of SFH_standard,electricity,2010,World,34.70,MJ/m2,none,ref. [2]: value compiled from ecoinvent v3.4 b...
22,manufacturing of SFH_standard,natural gas,2010,World,20.90,MJ/m2,none,ref. [2]: value compiled from ecoinvent v3.4 b...
23,manufacturing of SFH_standard,gasoline,2010,World,156.20,MJ/m2,none,ref. [2]: value compiled from ecoinvent v3.4 b...
35,manufacturing of MFH_standard,diesel,2010,World,5.00,MJ/m2,none,ref. [2]: value compiled from ecoinvent v3.4 b...
...,...,...,...,...,...,...,...,...
169,manufacturing of RT_standard,electricity,2010,World,0.36,MJ/m2,none,ref. [2]: value compiled from ecoinvent v3.4 b...
170,manufacturing of RT_standard,gasoline,2010,World,156.20,MJ/m2,none,ref. [2]: value compiled from ecoinvent v3.4 b...
171,manufacturing of RT_standard,diesel,2010,World,5.00,MJ/m2,none,ref. [2]: value compiled from ecoinvent v3.4 b...
172,manufacturing of RT_standard,electricity,2010,World,0.36,MJ/m2,none,ref. [2]: value compiled from ecoinvent v3.4 b...


Let's multiply the values assuming manufacturing of MFH and AB is the same. let's calculate manufacturing totals for all energy carriers separately: diesel, electricity, natural gas, gasoline 

In [20]:
pd.set_option('display.max_rows', None)

In [21]:
manufacturing_totals = total_heated_area.copy()

### DIESEL 

manufacturing_totals["diesel_energy"] = np.nan

# Loop through the rows of the dataframe
for idx, row in manufacturing_totals.iterrows():
    # Check if 'archetype' starts with 'SFH'
    if row["archetype"].startswith("SFH"):
        # Filter energy intensity data for SFH
        filtered_energy_int = manufacturing_energy_int[
            (manufacturing_energy_int["Energy_Carriers_m6"] == "diesel") & 
            (manufacturing_energy_int["Manufacturing_i3"] == "manufacturing of SFH_standard")
        ]
    elif row["archetype"].startswith("MFH"):
        # Filter energy intensity data for MFH or other archetypes
        filtered_energy_int = manufacturing_energy_int[
            (manufacturing_energy_int["Energy_Carriers_m6"] == "diesel") & 
            (manufacturing_energy_int["Manufacturing_i3"] == "manufacturing of MFH_standard")
        ]

    else: 
        filtered_energy_int = manufacturing_energy_int[
            (manufacturing_energy_int["Energy_Carriers_m6"] == "diesel") & 
            (manufacturing_energy_int["Manufacturing_i3"] == "manufacturing of RT_standard")
        ]
    
    # Ensure there is a valid entry in the filtered data
    if not filtered_energy_int.empty:
        # Multiply by the value in the 'total_heated_area' column
        manufacturing_totals.at[idx, "diesel_energy"] = row["total_heated_area"] * filtered_energy_int["value"].values[0]

# Final result
manufacturing_totals

,Unnamed: 0,kommunenum,archetype,total_heated_area,diesel_energy
0,0,3101,AB_1955,44912.0,224560.0
1,1,3103,AB_1955,36456.0,182280.0
2,2,3105,AB_1955,46032.0,230160.0
3,3,3107,AB_1955,46760.0,233800.0
4,4,3110,AB_1955,0.0,0.0
5,5,3112,AB_1955,0.0,0.0
6,6,3114,AB_1955,336.0,1680.0
7,7,3116,AB_1955,0.0,0.0
8,8,3118,AB_1955,15400.0,77000.0
9,9,3120,AB_1955,1456.0,7280.0


In [22]:
### ELECTRICITY

manufacturing_totals["electricity_energy"] = np.nan

# Loop through the rows of the dataframe
for idx, row in manufacturing_totals.iterrows():
    # Check if 'archetype' starts with 'SFH'
    if row["archetype"].startswith("SFH"):
        # Filter energy intensity data for SFH
        filtered_energy_int = manufacturing_energy_int[
            (manufacturing_energy_int["Energy_Carriers_m6"] == "electricity") & 
            (manufacturing_energy_int["Manufacturing_i3"] == "manufacturing of SFH_standard")
        ]
    elif row["archetype"].startswith("MFH"):
        # Filter energy intensity data for MFH or other archetypes
        filtered_energy_int = manufacturing_energy_int[
            (manufacturing_energy_int["Energy_Carriers_m6"] == "electricity") & 
            (manufacturing_energy_int["Manufacturing_i3"] == "manufacturing of MFH_standard")
        ]

    else: 

        filtered_energy_int = manufacturing_energy_int[
            (manufacturing_energy_int["Energy_Carriers_m6"] == "electricity") & 
            (manufacturing_energy_int["Manufacturing_i3"] == "manufacturing of RT_standard")
        ]
    
    # Ensure there is a valid entry in the filtered data
    if not filtered_energy_int.empty:
        # Multiply by the value in the 'total_heated_area' column
        manufacturing_totals.at[idx, "electricity_energy"] = row["total_heated_area"] * filtered_energy_int["value"].values[0]

In [23]:
### NATURAL GAS

manufacturing_totals["natural_gas_energy"] = np.nan

# Loop through the rows of the dataframe
for idx, row in manufacturing_totals.iterrows():
    # Check if 'archetype' starts with 'SFH'
    if row["archetype"].startswith("SFH"):
        # Filter energy intensity data for SFH
        filtered_energy_int = manufacturing_energy_int[
            (manufacturing_energy_int["Energy_Carriers_m6"] == "natural gas") & 
            (manufacturing_energy_int["Manufacturing_i3"] == "manufacturing of SFH_standard")
        ]
    elif row["archetype"].startswith("MFH"):
        # Filter energy intensity data for MFH or other archetypes
        filtered_energy_int = manufacturing_energy_int[
            (manufacturing_energy_int["Energy_Carriers_m6"] == "natural gas") & 
            (manufacturing_energy_int["Manufacturing_i3"] == "manufacturing of MFH_standard")
        ]

    else: 

        filtered_energy_int = manufacturing_energy_int[
            (manufacturing_energy_int["Energy_Carriers_m6"] == "natural gas") & 
            (manufacturing_energy_int["Manufacturing_i3"] == "manufacturing of RT_standard")
        ]
    
    # Ensure there is a valid entry in the filtered data
    if not filtered_energy_int.empty:
        # Multiply by the value in the 'total_heated_area' column
        manufacturing_totals.at[idx, "natural_gas_energy"] = row["total_heated_area"] * filtered_energy_int["value"].values[0]

In [24]:
### GASOLINE

manufacturing_totals["gasoline_energy"] = np.nan

# Loop through the rows of the dataframe
for idx, row in manufacturing_totals.iterrows():
    # Check if 'archetype' starts with 'SFH'
    if row["archetype"].startswith("SFH"):
        # Filter energy intensity data for SFH
        filtered_energy_int = manufacturing_energy_int[
            (manufacturing_energy_int["Energy_Carriers_m6"] == "gasoline") & 
            (manufacturing_energy_int["Manufacturing_i3"] == "manufacturing of SFH_standard")
        ]
    elif row["archetype"].startswith("MFH"):
        # Filter energy intensity data for MFH or other archetypes
        filtered_energy_int = manufacturing_energy_int[
            (manufacturing_energy_int["Energy_Carriers_m6"] == "gasoline") & 
            (manufacturing_energy_int["Manufacturing_i3"] == "manufacturing of MFH_standard")
        ]

    else: 

        filtered_energy_int = manufacturing_energy_int[
            (manufacturing_energy_int["Energy_Carriers_m6"] == "gasoline") & 
            (manufacturing_energy_int["Manufacturing_i3"] == "manufacturing of RT_standard")
        ]
        
    # Ensure there is a valid entry in the filtered data
    if not filtered_energy_int.empty:
        # Multiply by the value in the 'total_heated_area' column
        manufacturing_totals.at[idx, "gasoline_energy"] = row["total_heated_area"] * filtered_energy_int["value"].values[0]

In [26]:
pd.set_option('display.max_rows', 10)
manufacturing_totals

,Unnamed: 0,kommunenum,archetype,total_heated_area,diesel_energy,electricity_energy,natural_gas_energy,gasoline_energy
0,0,3101,AB_1955,44912.0,224560.0,16168.32,NaN,7015254.4
1,1,3103,AB_1955,36456.0,182280.0,13124.16,NaN,5694427.2
2,2,3105,AB_1955,46032.0,230160.0,16571.52,NaN,7190198.4
3,3,3107,AB_1955,46760.0,233800.0,16833.60,NaN,7303912.0
4,4,3110,AB_1955,0.0,0.0,0.00,NaN,0.0
...,...,...,...,...,...,...,...,...
8563,8563,5628,MFH_2021_,960.0,4800.0,345.60,NaN,149952.0
8564,8564,5630,MFH_2021_,0.0,0.0,0.00,NaN,0.0
8565,8565,5632,MFH_2021_,0.0,0.0,0.00,NaN,0.0
8566,8566,5634,MFH_2021_,0.0,0.0,0.00,NaN,0.0


In [27]:
print(f"Total manufacturing energy intensity using: \n"
      f"diesel is {manufacturing_totals['diesel_energy'].sum()} MJ, \n"
      f"electricity is {manufacturing_totals['electricity_energy'].sum()} MJ, \n"
      f"natural gas is {manufacturing_totals['natural_gas_energy'].sum()} MJ, \n"
      f"gasoline is {manufacturing_totals['gasoline_energy'].sum()} MJ")

Total manufacturing energy intensity using: 
diesel is 1590680141.0 MJ, 
electricity is 6207848096.0 MJ, 
natural gas is 3717868187.0 MJ, 
gasoline is 43024166141.0 MJ


In [28]:
total_manufact_emissions = {"diesel": manufacturing_totals['diesel_energy'].sum(),
                           "electricity": manufacturing_totals['electricity_energy'].sum(),
                           "natural gas": manufacturing_totals['natural_gas_energy'].sum(),
                           "gasoline": manufacturing_totals['gasoline_energy'].sum()}
manufacturing_emissions_df = pd.DataFrame(list(total_manufact_emissions.items()), columns=['Energy Carrier', 'Energy Intensity (MJ)'])
manufacturing_emissions_df

,Energy Carrier,Energy Intensity (MJ)
0,diesel,1.590680e+09
1,electricity,6.207848e+09
2,natural gas,3.717868e+09
3,gasoline,4.302417e+10


In [29]:
direct_emissions

,extensions,energy carrier,value,unit,stats_array_string,comment
0,CO2 emissions per main output,electricity,0.000,kg/MJ,none,"From Modaresi et al. (DOI 10.1021/es502930w), ..."
1,CO2 emissions per main output,heat,0.000,kg/MJ,none,"From Modaresi et al. (DOI 10.1021/es502930w), ..."
2,CO2 emissions per main output,"coal, hard coal",0.110,kg/MJ,none,"From Modaresi et al. (DOI 10.1021/es502930w), ..."
3,CO2 emissions per main output,"coal, lignite",0.110,kg/MJ,none,"From Modaresi et al. (DOI 10.1021/es502930w), ..."
4,CO2 emissions per main output,coke,0.110,kg/MJ,none,"From Modaresi et al. (DOI 10.1021/es502930w), ..."
...,...,...,...,...,...,...
7,CO2 emissions per main output,jet fuel,0.070,kg/MJ,none,"From Modaresi et al. (DOI 10.1021/es502930w), ..."
8,CO2 emissions per main output,gasoline,0.070,kg/MJ,none,"From Modaresi et al. (DOI 10.1021/es502930w), ..."
9,CO2 emissions per main output,natural gas,0.070,kg/MJ,none,"From Modaresi et al. (DOI 10.1021/es502930w), ..."
10,CO2 emissions per main output,hydrogen,0.000,kg/MJ,none,"From Modaresi et al. (DOI 10.1021/es502930w), ..."


In [30]:
manufacturing_emissions_df["Direct Emissions [kg]"] = np.nan

# Electricity intensity for electricity energy carrier
electricity_intensity = (31 / 1000) / 3.6

# Loop through the rows of the DataFrame to calculate emissions
for idx, row in manufacturing_emissions_df.iterrows():
    energy_carrier = row["Energy Carrier"]
    
    # Filter emissions based on energy carrier
    if energy_carrier == "diesel":
        filtered_emissions = direct_emissions[direct_emissions["energy carrier"] == "diesel"]
    elif energy_carrier == "natural gas":
        filtered_emissions = direct_emissions[direct_emissions["energy carrier"] == "natural gas"]
    elif energy_carrier == "gasoline":
        filtered_emissions = direct_emissions[direct_emissions["energy carrier"] == "gasoline"]
    elif energy_carrier == "electricity":
        # Apply electricity intensity directly for electricity
        manufacturing_emissions_df.at[idx, "Direct Emissions [kg]"] = row["Energy Intensity (MJ)"] * electricity_intensity
        continue  # Skip further processing if the energy carrier is electricity
    
    # If emissions data is found for diesel, natural gas, or gasoline
    if not filtered_emissions.empty:
        manufacturing_emissions_df.at[idx, "Direct Emissions [kg]"] = row["Energy Intensity (MJ)"] * filtered_emissions["value"].values[0]

manufacturing_emissions_df


,Energy Carrier,Energy Intensity (MJ),Direct Emissions [kg]
0,diesel,1.590680e+09,1.113476e+08
1,electricity,6.207848e+09,5.345647e+07
2,natural gas,3.717868e+09,2.602508e+08
3,gasoline,4.302417e+10,3.011692e+09


## Now let's calculate the material production direct emissions using the datasets

In [31]:
process_energy_int = process_energy_int[process_energy_int["Time"] == 2025]
process_energy_int

,Material_Production_i2,Energy_Carriers_m6,Time,SSP_Regions_1,value,unit,Stats_array_string,Comments
10,"production of copper electric grade, primary",electricity,2025,World,19.855443,MJ/kg,none,none
56,"production of copper electric grade, primary",natural gas,2025,World,4.873215,MJ/kg,none,none
102,production of cement,electricity,2025,World,0.327600,MJ/kg,none,none
148,production of cement,natural gas,2025,World,2.940000,MJ/kg,none,none
194,"production of concrete, aggregates",electricity,2025,World,0.008280,MJ/kg,none,none
...,...,...,...,...,...,...,...,...
2816,production of bricks from clay,electricity,2025,World,0.122400,MJ/kg,none,none
2862,production of bricks from clay,natural gas,2025,World,0.864000,MJ/kg,none,none
2908,production of bricks from clay,"coal, hard coal",2025,World,0.025000,MJ/kg,none,none
2954,production of bricks from clay,diesel,2025,World,0.030000,MJ/kg,none,none


In [32]:
cement_data = data[data["Material type"] == "cement"].copy()
cement_data["Energy_intensity_electr"] = np.nan
cement_data["Energy_intensity_nat_gas"] = np.nan

electr_filter = (process_energy_int["Material_Production_i2"] == "production of cement") & (process_energy_int["Energy_Carriers_m6"] == "electricity")
nat_gas_filter = (process_energy_int["Material_Production_i2"] == "production of cement") & (process_energy_int["Energy_Carriers_m6"] == "natural gas")


electr_value = process_energy_int.loc[electr_filter, "value"].values
nat_gas_value = process_energy_int.loc[nat_gas_filter, "value"].values


if len(electr_value) > 0:
    cement_data["Energy_intensity_electr"] = cement_data["total_material_amounts"] * electr_value[0]

if len(nat_gas_value) > 0:
    cement_data["Energy_intensity_nat_gas"] = cement_data["total_material_amounts"] * nat_gas_value[0]


In [33]:
concrete_data = data[(data["Material type"] == "concrete") | (data["Material type"] == "concrete surrogate")].copy()
concrete_data["Energy_intensity_electr"] = np.nan
concrete_data["Energy_intensity_diesel"] = np.nan

electr_filter = (process_energy_int["Material_Production_i2"] == "production of concrete, aggregates") & (process_energy_int["Energy_Carriers_m6"] == "electricity")
diesel_filter = (process_energy_int["Material_Production_i2"] == "production of concrete, aggregates") & (process_energy_int["Energy_Carriers_m6"] == "diesel")


electr_value = process_energy_int.loc[electr_filter, "value"].values
diesel_value = process_energy_int.loc[diesel_filter, "value"].values


if len(electr_value) > 0:
    concrete_data["Energy_intensity_electr"] = concrete_data["total_material_amounts"] * electr_value[0]

if len(diesel_value) > 0:
    concrete_data["Energy_intensity_diesel"] = concrete_data["total_material_amounts"] * diesel_value[0]


In [34]:
wood_data = data[(data["Material type"] == "wood and wood products") | (data["Material type"] == "wood surrogate")].copy()
wood_data["Energy_intensity_electr"] = np.nan
wood_data["Energy_intensity_diesel"] = np.nan

electr_filter = (process_energy_int["Material_Production_i2"] == "production of wood and wood products, primary") & (process_energy_int["Energy_Carriers_m6"] == "electricity")
diesel_filter = (process_energy_int["Material_Production_i2"] == "production of wood and wood products, primary") & (process_energy_int["Energy_Carriers_m6"] == "diesel")


electr_value = process_energy_int.loc[electr_filter, "value"].values
diesel_value = process_energy_int.loc[diesel_filter, "value"].values


if len(electr_value) > 0:
    wood_data["Energy_intensity_electr"] = wood_data["total_material_amounts"] * electr_value[0]

if len(diesel_value) > 0:
    wood_data["Energy_intensity_diesel"] = wood_data["total_material_amounts"] * diesel_value[0]

In [35]:
glass_data = data[data["Material type"] == "glass"].copy()
glass_data["Energy_intensity_electr"] = np.nan
glass_data["Energy_intensity_nat_gas"] = np.nan

electr_filter = (process_energy_int["Material_Production_i2"] == "production of glass, primary") & (process_energy_int["Energy_Carriers_m6"] == "electricity")
nat_gas_filter = (process_energy_int["Material_Production_i2"] == "production of glass, primary") & (process_energy_int["Energy_Carriers_m6"] == "natural gas")


electr_value = process_energy_int.loc[electr_filter, "value"].values
nat_gas_value = process_energy_int.loc[nat_gas_filter, "value"].values


if len(electr_value) > 0:
    glass_data["Energy_intensity_electr"] = glass_data["total_material_amounts"] * electr_value[0]

if len(nat_gas_value) > 0:
    glass_data["Energy_intensity_nat_gas"] = glass_data["total_material_amounts"] * nat_gas_value[0]


In [36]:
steel_data = data[data["Material type"] == "construction grade steel"].copy()
steel_data["Energy_intensity_electr"] = np.nan
steel_data["Energy_intensity_nat_gas"] = np.nan

electr_filter = (process_energy_int["Material_Production_i2"] == "production of construction grade steel, primary") & (process_energy_int["Energy_Carriers_m6"] == "electricity")
nat_gas_filter = (process_energy_int["Material_Production_i2"] == "production of construction grade steel, primary") & (process_energy_int["Energy_Carriers_m6"] == "natural gas")
coal_filter = (process_energy_int["Material_Production_i2"] == "production of construction grade steel, primary") & (process_energy_int["Energy_Carriers_m6"] == "coal, hard coal")

electr_value = process_energy_int.loc[electr_filter, "value"].values
nat_gas_value = process_energy_int.loc[nat_gas_filter, "value"].values
coal_value = process_energy_int.loc[coal_filter, "value"].values


if len(electr_value) > 0:
    steel_data["Energy_intensity_electr"] = steel_data["total_material_amounts"] * electr_value[0]

if len(nat_gas_value) > 0:
    steel_data["Energy_intensity_nat_gas"] = steel_data["total_material_amounts"] * nat_gas_value[0]

if len(coal_value) > 0:
    steel_data["Energy_intensity_coal"] = steel_data["total_material_amounts"] * coal_value[0]



In [37]:
# This dataset is necessary for insulation calculations since it has data on mineral wool insulation type

process_energy_int_2 = pd.read_excel("/Users/dolgayamaria/Thesis/Practical Part/Data/CURRENT_VN1_0_from_Lola/4_EI_ProcessEnergyIntensity_VN1.0.xlsx", sheet_name = "Values_Master")
process_energy_int_2 = process_energy_int_2[process_energy_int_2["Time"] == 2025]

In [38]:
insulation_data = data[data["Material type"] == "construction grade steel"].copy()
insulation_data["Energy_intensity_electr"] = np.nan
insulation_data["Energy_intensity_nat_gas"] = np.nan

electr_filter = (process_energy_int_2["Material_Production_i2"] == "production of insulation material") & (process_energy_int_2["Energy_Carriers_m6"] == "electricity")
nat_gas_filter = (process_energy_int_2["Material_Production_i2"] == "production of insulation material") & (process_energy_int_2["Energy_Carriers_m6"] == "natural gas")

electr_value = process_energy_int_2.loc[electr_filter, "value"].values
nat_gas_value = process_energy_int_2.loc[nat_gas_filter, "value"].values


if len(electr_value) > 0:
    insulation_data["Energy_intensity_electr"] = insulation_data["total_material_amounts"] * electr_value[0]

if len(nat_gas_value) > 0:
    insulation_data["Energy_intensity_nat_gas"] = insulation_data["total_material_amounts"] * nat_gas_value[0]


In [39]:
material_production = {
    "cement": {
        "electricity": cement_data["Energy_intensity_electr"].sum(),
        "natural gas": cement_data["Energy_intensity_nat_gas"].sum(),
        "diesel": 0, 
        "coal": 0
    },

    "cocrete": {
        "electricity": concrete_data["Energy_intensity_electr"].sum(),
        "natural gas": 0,
        "diesel": concrete_data["Energy_intensity_diesel"].sum(), 
        "coal": 0
    },

    "wood": {
        "electricity": wood_data["Energy_intensity_electr"].sum(),
        "natural gas": 0,
        "diesel": wood_data["Energy_intensity_diesel"].sum(), 
        "coal": 0
    },

     "glass": {
        "electricity": glass_data["Energy_intensity_electr"].sum(),
        "natural gas": glass_data["Energy_intensity_nat_gas"].sum() ,
        "diesel": 0, 
        "coal": 0
    },

    "grade steel": {
        "electricity": steel_data["Energy_intensity_electr"].sum(),
        "natural gas": steel_data["Energy_intensity_nat_gas"].sum(),
        "diesel": 0, 
        "coal": steel_data["Energy_intensity_coal"].sum()
    },

    "insulation": {
        "electricity": insulation_data["Energy_intensity_electr"].sum(),
        "natural gas": insulation_data["Energy_intensity_electr"].sum(),
        "diesel": 0, 
        "coal": 0
    }


}
material_production

{'cement': {'electricity': 1314081695.6512663,
  'natural gas': 11793040858.408802,
  'diesel': 0,
  'coal': 0},
 'cocrete': {'electricity': 1513792035.2703252,
  'natural gas': 0,
  'diesel': 4199675947.124955,
  'coal': 0},
 'wood': {'electricity': 26059724434.986317,
  'natural gas': 0,
  'diesel': 5490716170.846702,
  'coal': 0},
 'glass': {'electricity': 366178394.0318433,
  'natural gas': 7458475710.01702,
  'diesel': 0,
  'coal': 0},
 'grade steel': {'electricity': 3717432052.49601,
  'natural gas': 292850622.05672127,
  'diesel': 0,
  'coal': 185555596275.86517},
 'insulation': {'electricity': 63223151286.886246,
  'natural gas': 63223151286.886246,
  'diesel': 0,
  'coal': 0}}

In [40]:
material_production_df = pd.DataFrame.from_dict(material_production, orient="index")
material_production_df

,electricity,natural gas,diesel,coal
cement,1.314082e+09,1.179304e+10,0.000000e+00,0.000000e+00
cocrete,1.513792e+09,0.000000e+00,4.199676e+09,0.000000e+00
wood,2.605972e+10,0.000000e+00,5.490716e+09,0.000000e+00
glass,3.661784e+08,7.458476e+09,0.000000e+00,0.000000e+00
grade steel,3.717432e+09,2.928506e+08,0.000000e+00,1.855556e+11
insulation,6.322315e+10,6.322315e+10,0.000000e+00,0.000000e+00
